In [40]:
#KLUE QA Dataset과 Roberta-large모델을 활용한 학습 예제 입니다.

import json
import random
import collections

import torch
import numpy as np
from tqdm.notebook import tqdm
from torch.utils.data import Dataset,DataLoader
from transformers import AutoModel, AutoTokenizer
from transformers import AutoTokenizer,AutoModelForQuestionAnswering,AdamW

In [41]:
with open("./dataset/klue-mrc-v1.1_train.json", 'rb') as f:
    klue_dict = json.load(f)

In [42]:
klue_dict["data"][0]

{'title': '제주도 장마 시작 … 중부는 이달 말부터',
 'paragraphs': [{'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.',
   'qas': [{'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
     'answers': [{'text': '한 달가량', 'answer_start': 478},
      {'text': '한 달', 'answer_start': 478}],
     'question_type': 1,
     'is_impos

In [43]:
#data를 읽어서 각 feature 별로 전처리하는 함수
def readData(path):
    with open(path,'rb')as file:
        MRCdata=json.load(file)
    #data 구성
    #context - 문장
    #question - 질문
    #answer - 정답
    
    contexts=list()
    
    questions=list()
    answers=list()
    
    #하나의 문장에 여러 질문이 있을 수 있고, 질문이 여러개면 답변도 여러개이기 때문에 4중for문 사용
    for item in tqdm(MRCdata["data"]):
        for passage in item['paragraphs']:
            context=passage['context']
            for qa in passage['qas']:
                question=qa['question']
                for ans in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(ans)
    return contexts,questions,answers

In [44]:
#dataset에 정답의 끝나는 index도 추가하는 함수
def endIdx(answers,contexts):
    for answer,context in zip(answers,contexts):
        ansText=answer['text']
        startIdx=answer['answer_start']
        endIdx=startIdx+len(ansText)
        
        answer['answer_end']=endIdx
        

In [45]:
#토크나이징, roberta-large사용
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

In [74]:
class KlueMRCDataset(Dataset):
    
    def __init__(self, contexts, questions, answers, modelMaxPositionEmbedings, tokenizer):
        self.tokenizer = tokenizer
        self.answers = answers
        self.questions = questions
        self.contexts = contexts
        self.model_max_position_embedings = modelMaxPositionEmbedings
        self.encodings = self.tokenizer(self.contexts, 
                                        self.questions,
                                        max_length=512,
                                        truncation=True,
                                        padding="max_length",
                                        return_token_type_ids=False)
        self.addTokenPositions()
        
    def addTokenPositions(self):
        startPositions = []
        endPositions = []
        for i in range(len(self.answers)):
            startPositions.append(self.encodings.char_to_token(i, self.answers[i]['answer_start']))
            endPositions.append(self.encodings.char_to_token(i, self.answers[i]['answer_end'] - 1))

            # positions 값이 None 값이라면, answer가 포함된 context가 잘렸다는 의미
            if startPositions[-1] is None:
                startPositions[-1] = self.model_max_position_embedings
            if endPositions[-1] is None:
                endPositions[-1] = self.model_max_position_embedings

        self.encodings.update({'startPositions': startPositions, 'endPositions': endPositions})

        
    def get_data(self):
        return {"contexts":self.contexts, 'questions':self.questions, 'answers':self.answers}
    
    
    def get_encodings(self):
        return self.encodings
        
    
    def __getitem__(self, idx):
        return {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
    
    def __len__(self):
        return len(self.encodings['input_ids'])

In [70]:
#훈련 data 전처리
contexts,questions,answers=readData("./dataset/klue-mrc-v1.1_train.json")
endIdx(answers,contexts)
trainDataset=KlueMRCDataset(contexts,questions,answers,512,tokenizer)

  0%|          | 0/12174 [00:00<?, ?it/s]

In [90]:
#model 가저오기
model = AutoModelForQuestionAnswering.from_pretrained("klue/bert-base")

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model chec

In [91]:
#하이퍼파라미터 정의
EPOCH=3
LEARNING_RATE=5e-5
BATCH_SIZE=8

In [94]:
#모델 훈련 실행함수(AdamW사용)
def train_runner(model, dataset, batch_size, num_train_epochs, learning_rate):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    model.to(device)
    model.train()
    train_dataloader = DataLoader(dataset=dataset, batch_size=batch_size)
    global_total_step = len(train_dataloader) * num_train_epochs
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0)
    with tqdm(total=global_total_step, unit='step') as t:
        total = 0
        total_loss = 0
        for epoch in range(num_train_epochs):
            for batch in train_dataloader:
                optimizer.zero_grad()
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                startPositions = batch['startPositions'].to(device)
                endPositions = batch['endPositions'].to(device)
                outputs = model(input_ids,
                             attention_mask=attention_mask,
                             start_positions=startPositions,
                             end_positions=endPositions)
                loss = outputs.loss
                loss.backward()
                optimizer.step()
                
                batch_loss = loss.item() * len(input_ids)
                total += len(input_ids)
                total_loss += batch_loss
                global_total_step += 1
                t.set_postfix(loss="{:.6f}".format(total_loss / total), batch_loss="{:.6f}".format(batch_loss))
                t.update(1)
                
                del input_ids
                del attention_mask
                del startPositions
                del endPositions
                del outputs
                del loss
    model.save_pretrained("./outputs/klue_output_model")


In [ ]:
#모델 훈련 실행
train_runner(model,trainDataset, BATCH_SIZE, EPOCH, LEARNING_RATE)

  0%|          | 0/6624 [00:00<?, ?step/s]